Um modelo de classificação de churn identificou que 2450 clientes têm probabilidade maior que 87% de
serem churn na próxima semana.

Este modelo possui uma precisão de 33%. Em cada 3 pessoas atendidas 2 não são churn e 1 é churn.
Para cada erro ( pessoa não tem interesse em receber uma proposta de refinanciamento ), ocorre um
custo médio de R$100,00

**Qual cliente deve ser priorizado na fila de atendimento?**

Cada cliente só pode ser atendido por um ponto de atendimento.

Cada atendente tem uma jornada diária de 6 horas (Segunda a Sexta) , sendo que a cada 90 minutos de
atendimento, o atendente tem direito a um descanso de 10 minutos

``` {python}
PA1 = 8
PA2 = 10
PA3 = 12
PA4 = 10

PA1 = 9 - 15
PA2 = 12 - 18
PA3 = 9 - 15
PA4 = 12 - 18
```

2450 clientes

5%  - 30000          - 1.5%  a.m   
10% - 20000 - 30000  - 1.6%  a.m   
20% - 10000 - 20000  - 1.65% a.m   
65% - 500   - 10000  - 1.70% a.m    

Prazo para financiamento é 72 parcelas.

Clientes do sexo feminino atendem mais o
telefone à tarde e na primeira metade da
semana (segunda, terça e quarta).

Clientes do sexo masculino atendem mais o
telefone pela manhã e na segunda metade da
semana (quarta, quinta e sexta).

60% dos clientes são do sexo feminino.
40% dos clientes são do sexo masculino.

30% dos clientes não atendem o telefone ou não
tem interesse em ouvir a proposta.

Para cada cliente que não quer ouvir a proposta,
perde-se 30 segundos em média.


In [1]:
import pandas as pd
import numpy as np

class Client:
    
    def __init__(self,client_id,value, gender,rate,age,installments,churn_prob):
        self.client_id = client_id
        self.value = value
        self.gender = gender
        self.rate   = rate
        self.age    = age
        self.installments = installments
        self.churn_prob = churn_prob
        
    def __repr__(self):
        return "Client Id:{0}, Value: {1} , Gender: {2}, Rate: {3}, Age: {4},Installments : {5}, Churn Prob: {6}" \
    .format(self.client_id,self.value,self.gender,self.rate, self.age,self.installments, self.churn_prob)

class Clients:
    
    def __init__(self):
        self.clients = []
        clientes = pd.read_csv("clients.csv")
        
        for index, row in clientes.iterrows():
            c = Client(row['client_id'],row['value'],row['gender'],row['rate'],row['age'],row['installments'],row['churn_prob'])
            self.addClient(c)
     
    def addClient(self,client):
        self.clients.append(client)
        
    def nclients(self):
        return len(self.clients)
    

In [2]:
clients = Clients()

In [3]:
clients.clients[1]

Client Id:1.0, Value: 6438.02 , Gender: 1.0, Rate: 1.7, Age: 75.0,Installments : 36.0, Churn Prob: 0.91

In [11]:
def max_atendimentos(avg, tempo_total):
    return int(tempo_total/avg)
    
    
pa1 = max_atendimentos(8,330)
pa2 = max_atendimentos(10,330)
pa3 = max_atendimentos(12,330)
pa4 = max_atendimentos(10,330)

print(pa1 , pa2, pa3, pa4)

41 33 27 33


In [12]:
def max_atendimentos_residuos(n,t):
    return int(((n * t) - (n - (n * 0.7)) * 0.5)/t + (n * 0.3))

m1 = max_atendimentos_residuos(pa1,8)
m2 = max_atendimentos_residuos(pa2,10)
m3 = max_atendimentos_residuos(pa3,12)
m4 = max_atendimentos_residuos(pa4,10)

print(m1, m2, m3 , m4)

52 42 34 42


### OTIMIZAÇÃO

In [13]:
from mip.model import Model, xsum, maximize
from mip.constants import BINARY

model = Model('fila-atendimento')

model.clear()
A = 4 
C = clients.nclients()
D = 5 

x = [[[model.add_var(name='x({},{},{})'.format(d, c, a), var_type=BINARY) for a in range(A)] for c in range(C)] for d in range(D)] # cliente, atendente , day 

In [14]:
# cada cliente só pode ser atendido por um atendente
for c in range(C):
    model += xsum(x[d][c][a] for a in range(A) for d in range(D)) <= 1 
    

for d in range(D):
    model += xsum(x[d][c][0] for c in range(C)) <= 41 + 11 
    model += xsum(x[d][c][1] for c in range(C))  <= 33 + 9 
    model += xsum(x[d][c][2] for c in range(C))  <= 27 + 7 
    model += xsum(x[d][c][3] for c in range(C))  <= 33 + 9 
    

for d in range(D):
    for a in range(A):
        model +=  xsum(x[d][c][a] for c in range(C)) >= 20
        model +=  xsum(x[d][c][a] for c in range(C)) >= 20
        model +=  xsum(x[d][c][a] for c in range(C)) >= 20
        model +=  xsum(x[d][c][a] for c in range(C)) >= 20

    
# ##  add age constraint 
for d in range(D-1):
    model +=  xsum(clients.clients[c].age * x[d][c][0] for c in range(C)) <= xsum(clients.clients[c].age * x[d+1][c][0] for c in range(C) )
    model +=  xsum(clients.clients[c].age * x[d][c][1] for c in range(C)) <= xsum(clients.clients[c].age * x[d+1][c][1] for c in range(C) )
    model +=  xsum(clients.clients[c].age * x[d][c][2] for c in range(C)) <= xsum(clients.clients[c].age * x[d+1][c][2] for c in range(C) )
    model +=  xsum(clients.clients[c].age * x[d][c][3] for c in range(C)) <= xsum(clients.clients[c].age * x[d+1][c][3] for c in range(C) )

    
for a in range(A):
    model +=  xsum( clients.clients[c].gender + x[0][c][a] for c in range(C) ) <= xsum(clients.clients[c].gender + x[0][c][a] for c in range(C))  
    model +=  xsum( clients.clients[c].gender + x[1][c][a] for c in range(C) ) <= xsum(clients.clients[c].gender + x[1][c][a] for c in range(C))  
    model +=  xsum( clients.clients[c].gender + x[2][c][a] for c in range(C) ) <= xsum(clients.clients[c].gender + x[2][c][a] for c in range(C)) 

    model +=  xsum( (1 - clients.clients[c].gender ) * x[3][c][a] for c in range(C) ) >= xsum(clients.clients[c].gender * x[3][c][a] for c in range(C)) 
    model +=  xsum( (1 - clients.clients[c].gender ) * x[4][c][a] for c in range(C) ) >= xsum(clients.clients[c].gender * x[4][c][a] for c in range(C)) 

In [15]:
import numpy as np

## linearizar a exponencial ?
def juros_compostos(C,i,t):
    return (C*(1+(t/100))**i)

def juros_compostos2(C,i,t):
    return np.log(C)+i*np.log(1+(t/100))

In [16]:
# model.objective = maximize(
#     1/3 * xsum(
#         juros_compostos( clients.clients[c].value,clients.clients[c].installments, clients.clients[c].rate ) * x[d][c][a] for c in range(C) for a in range(A) for d in range(D)
#         ) - 2/3 * xsum(
#              100 * x[d][c][a] for c in range(C) for a in range(A) for d in range(D)
#         )
# )
    
model.objective = maximize(
        xsum(
            juros_compostos2( clients.clients[c].value,clients.clients[c].installments, clients.clients[c].rate ) * x[d][c][a] for c in range(C) for a in range(A) for d in range(D)
        ))

    
#model.optimize()
# colocar prazo na função objetivo

#model.max_gap = 0.05

# model.clique = 2,
# model.cut_passes = 7

# if set to 1 (FEASIBILITY) then the search process will
# focus on try to find quickly feasible solutions and improving them; if set to 2 (OPTIMALITY)
# then the search process will try to find a provable optimal solution
model.emphasis = 2

model.optimize(max_seconds=600)

<OptimizationStatus.OPTIMAL: 0>

In [17]:
clientes_atendidos = 0
for a in range(A):
    for d in range(D):
        selected = [i for i in range(C) if x[d][i][a].x >= 0.99]
        logvalue = round(sum(juros_compostos2(clients.clients[s].value,clients.clients[s].installments, clients.clients[s].rate ) for s in selected),2)
        value = round(sum(juros_compostos(clients.clients[s].value,clients.clients[s].installments, clients.clients[s].rate ) for s in selected),2)*0.7*1/3
        print("{} + {}".format(logvalue,value))
        mean_age = sum(clients.clients[s].age for s in selected) / len(selected)
        male = len(selected)  - sum(clients.clients[s].gender for s in selected)
        female =  len(selected) - male
        print('Atendente {}: day {} clients: {} , Male: {} , Female: {} , value {}, sequence {} , Mean Age : {}'.format(a,d,len(selected),male, female, value,selected,mean_age))
        
        for c in selected:
            print(clients.clients[c])
        print('-'*30)
        clientes_atendidos += len(selected)

print(clientes_atendidos)

570.78 + 813644.1553333332
Atendente 0: day 0 clients: 52 , Male: 21.0 , Female: 31.0 , value 813644.1553333332, sequence [11, 146, 155, 179, 202, 246, 254, 259, 309, 348, 390, 415, 539, 630, 645, 698, 732, 768, 840, 856, 896, 909, 951, 1025, 1042, 1099, 1140, 1186, 1227, 1299, 1336, 1348, 1361, 1447, 1492, 1518, 1522, 1553, 1616, 1623, 1716, 1830, 1926, 2014, 2022, 2029, 2139, 2291, 2303, 2329, 2335, 2368] , Mean Age : 63.86538461538461
Client Id:11.0, Value: 33333.82 , Gender: 1.0, Rate: 1.5, Age: 62.0,Installments : 72.0, Churn Prob: 0.97
Client Id:146.0, Value: 18700.52 , Gender: 1.0, Rate: 1.65, Age: 68.0,Installments : 48.0, Churn Prob: 0.94
Client Id:155.0, Value: 48627.32 , Gender: 1.0, Rate: 1.5, Age: 66.0,Installments : 72.0, Churn Prob: 0.99
Client Id:179.0, Value: 19258.04 , Gender: 0.0, Rate: 1.65, Age: 72.0,Installments : 48.0, Churn Prob: 0.99
Client Id:202.0, Value: 63329.22 , Gender: 1.0, Rate: 1.5, Age: 58.0,Installments : 72.0, Churn Prob: 0.89
Client Id:246.0, Value

In [ ]:
https://medium.com/opex-analytics/machine-learning-using-mixed-integer-programming-af95e4d56863